In [ ]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('texts/input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

#Here’s what makes a RNN recurrent: it uses the same weights for each step. More specifically, 
#a typical vanilla RNN uses only 3 sets of weights to perform its calculations:
#Wxh​, used for all x_t​ → h_t​ links.
#Whh​, used for all h_{t-1}​ → h_t​ links.
#Why​, used for all h_t​ → y_t​ links.

Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output


#We’ll also use two biases for our RNN:
#b_h, added when calculating h_t.
#b_y, added when calculating y_t.
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

the format of output required to calculate the loss should be one-hot encoded vectors.
<img src="https://miro.medium.com/max/684/1*JXpSMnwL7Hx1cGny310DYg.png">

In [31]:
# Transform the result in probabilities for the next chars
def softmax(xs):
	return np.exp(xs) / np.sum(np.exp(xs))

In [74]:
#The np.tanh function implements a non-linearity that squashes the activations to the range [-1, 1]
#Notice briefly how this works: There are two terms inside of the tanh: one is based on the previous hidden state and one is based on the current input. 
#In numpy np.dot is matrix multiplication. The two intermediates interact with addition, and then get squashed by the tanh into the new state vector. 
#If you’re more comfortable with math notation, we can also write the hidden state update as ht=tanh(Whhht−1+Wxhxt), where tanh is applied elementwise.
def tanh_activation(input_to_hidden_weight, previous_hidden_state, hidden_to_hidden_weight, current_hidden_state,  bh):
	return np.tanh(np.dot(input_to_hidden_weight, previous_hidden_state) + np.dot(hidden_to_hidden_weight, current_hidden_state) + bh)


In [89]:
def cross_entropy_loss(ps, target):
	return -np.log(ps[target,0])

In [83]:
#The forward pass use the parameters of the model (Wxh, Whh, Why, bh, by) to calculate the next char given a char from the trainning set.
def forward(inputs, targets, hprev):
	xs, hs, ys, ps = {}, {}, {}, {}
	hs[-1] = np.copy(hprev)

	for t in range(len(inputs)):
		#xs[t] is the vector that encode the char at position t 
		xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
		xs[t][inputs[t]] = 1
		activation = tanh_activation(Wxh, xs[t],Whh, hs[t-1], bh) # hidden state
		# taking the dot product between the activation and the weight matrix -- this is called the "net input" to the current layer
		net = np.dot(Why, activation) + by
		prediction = softmax(net) # probabilities for next chars
		hs[t] = activation
		ys[t] = net
		ps[t] = prediction
	return xs, hs, ys, ps


In [94]:
def tanh_deriv(activationFuncResult, dh):
	return (1 - activationFuncResult * activationFuncResult) * dh 

In [107]:
def backprop(inputs, targets, activations, predictions, xs):
	# backward pass: compute gradients going backwards
	dWxh = np.zeros_like(Wxh)
	dWhh = np.zeros_like(Whh)
	dWhy = np.zeros_like(Why)
	dbh = np.zeros_like(bh)
	dby = np.zeros_like(by)
	dhnext = np.zeros_like(activations[0])

	for t in reversed(range(len(inputs))):
		prediction = np.copy(predictions[t])
		prediction[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
		dWhy += np.dot(prediction, activations[t].T)
		dh = np.dot(Why.T, prediction) + dhnext # backprop into h
		direction = tanh_deriv(activations[t], dh) # backprop through tanh nonlinearity
		dWxh += np.dot(direction, xs[t].T)
		dWhh += np.dot(direction, activations[t-1].T)
		dhnext = np.dot(Whh.T, direction)
		dby += prediction    
		dbh += direction
    
	for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
		np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    
	return  dWxh, dWhh, dWhy, dbh, dby, activations[len(inputs)-1]


In [104]:
def sample(previous_hidden_state, start_index, text_length):
	""" 
	sample a sequence of integers from the model 
	h is memory state, seed_ix is seed letter for first time step
	"""
	x = np.zeros((vocab_size, 1))
	x[start_index] = 1
	indexes = []
	for t in range(text_length):
		h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, previous_hidden_state) + bh)
		y = np.dot(Why, previous_hidden_state) + by
		p = np.exp(y) / np.sum(np.exp(y))
		index = np.random.choice(range(vocab_size), p=p.ravel())
		x = np.zeros((vocab_size, 1))
		x[index] = 1
		indexes.append(index)
	return indexes


In [61]:
sample_ix = sample(hprev, inputs[0], 200)
''.join(ix_to_char[ix] for ix in sample_ix)

'ntp  htacte   lp Ccwsscgrs  bsd ttie ettisnat  e etsc a et li  ethiepetae  cev tt yu.lityticplia?y  lratitidspe a tsd dse di  yes  bigtsitoyresyab esk !htiRatad rtc rtttbttti csdekepcil te bs s dtwnye'

In [129]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
epochs = 500000

In [127]:
# perform parameter(weights) update with Adagrad
def update_weights(dWxh, dWhh, dWhy, dbh, dby):
	for weights, dWeights, memWeights in zip([Wxh, Whh, Why, bh, by], [dWxh, dWhh, dWhy, dbh, dby], [mWxh, mWhh, mWhy, mbh, mby]):
		memWeights += dWeights * dWeights
		weights += -learning_rate * dWeights / np.sqrt(memWeights + 1e-8) # adagrad update


In [133]:
def calculate_loss(ps, targets):
	loss = 0
	for t in range(len(inputs)):
		loss += cross_entropy_loss(ps[t],targets[t])
        
	return smooth_loss * 0.999 + loss * 0.001

In [ ]:
for n in np.arange(0, epochs):
	# prepare inputs (we're sweeping from left to right in steps seq_length long)
	if p+seq_length+1 >= len(data) or n == 0: 
		hprev = np.zeros((hidden_size,1)) # reset RNN memory
		p = 0 # go from start of data

	inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
	targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
	xs, hs, ys, ps = forward(inputs, targets, hprev)
	dWxh, dWhh, dWhy, dbh, dby, hprev = backprop(inputs, targets, hs, ps, xs)    
	smooth_loss = calculate_loss(ps, targets)
	update_weights(dWxh, dWhh, dWhy, dbh, dby)
	p += seq_length # move data pointer
	# sample from the model now and then
	if n % 100 == 0:
		sample_ix = sample(hprev, inputs[0], 200)
		txt = ''.join(ix_to_char[ix] for ix in sample_ix)
		print ('----\n %s \n----' % (txt, ))
		print ('iter %d, loss: %f' % (n, smooth_loss))


----
    .ue   s e. es ,s    shsae  s ne e es.h e  .s  esuh e  h,ai u   .sshrh h.
e is  iy    e.ehek  s h   .lh
 e     esh eas se e . ycsS.c ehi;si  u eeieh.e .oe  yee,e,ees i   eli siiiRu eeee, sahly ee. h 
----
iter 0, loss: 55.491125
----
 ppSupwwtnwpwtpnpnwwwnnpidnnwkpnpwmwwwwpiidwimptwpnppwrmwtpnvwdpdwpdwintwUsddpc vtmnnpypmepwpintpicdelwtnipppvpinwPpipwuwbnwttnRnwunwtwntnwtpinwwppwwtipvwwrpwdwnwwtpputwmi.wmwwwpnwpttdpwpwnntpwppinwwng 
----
iter 100, loss: 55.426974
----
 iith.hy.i i.  i.top     hy o o.. . ehhth i h,. .tt.     . i  thtthit..i. tht.eh th   .t  a.  h ..h . ih.  t .i h i y  .i  t.h h tt    t h,hh .ht t .   ipi .ij ...i.i t  i t.httt   t .. ti.hh  t  tt    
----
iter 200, loss: 55.463700
----
 tchi .t                  hi       h  ti         t     h t         t h  .         t t    c         t   t   i. h        , tc      i    t    .  t   i         h ii t       . t     h   t              t     
----
iter 300, loss: 55.500530
----
 ItatlrUlbPtlwt(ottcngobts sbpatdgtt.tPsx